<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_%E2%84%96_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🧪 **Лабораторный практикум № 4**  
## **Сравнительный анализ ML-моделей для прогнозирования временных рядов**

**Кафедра:** Кафедра анализа данных и технологий программирования  
**Дисциплина:** Машинное обучение в задачах прогнозирования  
**Уровень:** Магистратура, 2 курс  
**Преподаватель:** Арабов Муллошараф Курбонович  
**Консультации:** 21.09.2025  

---

## 🎯 1. Цели и задачи работы

### **Цель:**  
Освоить **машинно-обучающий подход к прогнозированию временных рядов**, включая автоматизированные платформы (AutoGluon-TimeSeries), традиционные регрессионные и ансамблевые модели, а также современные методы валидации, инжиниринга признаков и интерпретации. Научиться **сравнивать автоматизированный и кастомный подходы** с учётом качества, скорости и интерпретируемости.

### **Задачи:**
1. Реализовать **расширенный feature engineering**: лаги, скользящие статистики, временные признаки, сезонные компоненты Фурье, экспоненциальное сглаживание.
2. Применить **целевые трансформации**: лог-трансформация, **преобразование Бокса–Кокса**.
3. Разделить данные на **train/val/test (60%/20%/20%)** в хронологическом порядке.
4. Построить и сравнить следующие модели:
   - **Линейные**: Linear, Ridge, Lasso, Elastic Net, Bayesian Regression  
   - **Ансамбли**: Random Forest, Extra Trees, XGBoost, LightGBM, CatBoost, Stacking  
   - **SVM**: SVR, NuSVR  
   - **Автоматизированная платформа**: **AutoGluon-TimeSeries** с пресетами `medium_quality`, `high_quality`, `best_quality`  
   - **Бейзлайны**: Naive, Seasonal Naive, Moving Average, Linear Trend  
5. Протестировать **четыре стратегии прогнозирования**: Direct, Recursive, Multi-output, DirRec.
6. Провести **валидацию по времени**:  
   - `TimeSeriesSplit` (расширяющееся окно),  
   - Purged Walk-Forward,  
   - Групповая блочная валидация,  
   - Встроенная backtesting-валидация AutoGluon.
7. Выполнить **подбор гиперпараметров** через:
   - `GridSearchCV` и `RandomizedSearchCV` с временной CV,  
   - **Optuna** с Bayesian optimization,  
   - Встроенную AutoML-оптимизацию AutoGluon.
8. Провести **диагностику моделей**:
   - Анализ остатков (ACF, стационарность),  
   - Feature importance,  
   - Partial Dependence Plots,  
   - Визуализация ошибок по времени.
9. Оценить качество по **масштабированным и бизнес-ориентированным метрикам**, включая **тест Diebold–Mariano**.
10. Сравнить **AutoGluon vs кастомные модели** по качеству, скорости и гибкости.
11. Разработать **рекомендации по применению в продакшене** и **шаблон переобучения**.

> 💡 **Важно:** Работа фокусируется на **машинно-обучающих моделях** и **автоматизации** — классические статистические модели (ARIMA, Prophet и др.) **не рассматриваются**.

> 📌 **Требования к данным:**  
> - Объём: **≥500 наблюдений**  
> - Горизонты прогноза: **h ∈ {1, 7, 30}**  
> - Наличие временной метки и целевой переменной  

---

## 📚 2. Теоретические предпосылки

В отличие от классических моделей, ML-подходы **не требуют предположений о стационарности**, но **чувствительны к корректному инжинирингу признаков** и **утечке данных во времени**.

- **Линейные модели** служат базовым бейзлайном и обладают высокой интерпретируемостью.
- **Деревья и градиентный бустинг** эффективны при нелинейных зависимостях и взаимодействиях фичей.
- **AutoGluon-TimeSeries** — современная AutoML-платформа, автоматически:
  - Генерирует признаки,
  - Подбирает гиперпараметры,
  - Строит ансамбли (включая DeepAR, TFT, SimpleFeedForward),
  - Проводит backtesting.
- **Стратегии прогнозирования** определяют, как модель обрабатывает многопшаговую задачу:
  - *Direct*: устойчива к накоплению ошибки, но требует много моделей.
  - *Recursive*: элегантна, но склонна к дрейфу.
- **Валидация по времени** — единственный корректный способ оценки без утечки будущего.

> 🔑 **Ключевые инструменты**:  
> - `scikit-learn` (леса, SVM, линейные модели)  
> - `xgboost`, `lightgbm`, `catboost`  
> - `optuna` (Bayesian optimization)  
> - `autogluon.timeseries` (AutoML)  
> - `sklearn.model_selection.TimeSeriesSplit`  
> - `statsmodels.stats.diagnostic.dm_test` (Diebold–Mariano)

---

## 🧪 3. Методика эксперимента и порядок выполнения

### **3.1. Этап 1. Инжиниринг признаков**

- **Лаги**: 1, 2, 3, 7, 14, 30  
- **Скользящие окна**:
  - mean, std, min, max, median → окна 7, 14, 30  
- **Экспоненциальное сглаживание**:
  - `α = 0.3, 0.5, 0.7` → `y_t^α = α·y_t + (1−α)·y_{t−1}^α`  
- **Временные признаки**:
  - `day_of_week`, `month`, `quarter`, `week_of_year`  
  - `is_holiday`, `is_weekend` (если доступно)  
- **Сезонные компоненты**:
  - Fourier terms: `sin(2π·t/s)`, `cos(2π·t/s)` для s = 7, 30  
- **Целевые трансформации**:
  - Логарифм (если y > 0)  
  - **Бокса–Кокса**: `λ = boxcox_normmax(y_train)` → обратное преобразование при оценке

---

### **3.2. Этап 2. Валидация и разбиение данных**

- **Хронологическое разбиение**:  
  - train: 60%  
  - val: 20% (для гиперпараметров)  
  - test: 20% (финальная оценка)  
- **TimeSeriesSplit** (n_splits=5, max_train_size=365)  
- **Purged Walk-Forward**: gap между train и test, чтобы избежать утечки  
- **AutoGluon backtesting**:  
  ```python
  predictor = TimeSeriesPredictor(
      target="y", prediction_length=7, eval_metric="MAE"
  ).fit(
      train_data,
      presets=["medium_quality", "high_quality", "best_quality"],
      include_models=["DeepAR", "TemporalFusionTransformer", "SimpleFeedForward"]
  )
  ```

---

### **3.3. Этап 3. Подбор гиперпараметров**

- **GridSearchCV** с `TimeSeriesSplit` для линейных моделей  
- **Optuna** для градиентного бустинга:
  ```python
  def objective(trial):
      params = {"n_estimators": trial.suggest_int("n_estimators", 100, 1000), ...}
      model = LGBMRegressor(**params)
      scores = cross_val_score(model, X_train, y_train, cv=tscv, scoring="neg_mean_absolute_error")
      return -scores.mean()
  study = optuna.create_study(direction="minimize")
  study.optimize(objective, n_trials=100)
  ```
- **AutoGluon**: полная автоматизация подбора + ансамблирования

---

### **3.4. Этап 4. Стратегии прогнозирования**

- **Direct**: обучение `h` моделей → `ŷ_{t+1}, ..., ŷ_{t+h}`  
- **Recursive**: одна модель → `ŷ_{t+1} → вход → ŷ_{t+2}`  
- **Multi-output**: одна модель предсказывает вектор `[ŷ_{t+1}, ..., ŷ_{t+h}]`  
- **DirRec**: гибрид — рекурсивная в пределах окна, прямая между окнами  
- **AutoGluon**: поддерживает **multi-horizon forecasting** «из коробки»

> Сравнить по **накоплению ошибки**, **времени обучения**, **гибкости**.

---

### **3.5. Этап 5. Построение моделей**

| Группа | Модели | Особенности |
|-------|--------|-------------|
| **Линейные** | Linear, Ridge, Lasso, ElasticNet, BayesianRidge | Интерпретируемость, устойчивость |
| **Ансамбли** | RF, ExtraTrees, XGBoost, LightGBM, CatBoost, Stacking | Нелинейность, высокая точность |
| **SVM** | SVR, NuSVR | Чувствительны к масштабу, медленны на больших данных |
| **AutoML** | AutoGluon-TimeSeries | DeepAR, TFT, ансамбли, автоматизация |
| **Бейзлайны** | Naive, SNaive, MA, Linear Trend | Нижняя граница качества |

---

### **3.6. Этап 6. Диагностика моделей**

- **Остатки**: ACF, ADF/KPSS — есть ли структура?  
- **Feature importance**: SHAP или встроенные веса (для деревьев)  
- **PDP**: как изменяется прогноз при изменении фичи?  
- **Визуализация**:  
  - Факт vs прогноз на test  
  - Ошибки по времени (где модель «падает»)  
  - Доверительные интервалы (для AutoGluon)  
- **AutoGluon leaderboard**: сравнение внутренних моделей

---

### **3.7. Этап 7. Оценка качества**

- **Метрики**: MAE, RMSE, MAPE, **MASE**, **RMSSE** (масштабированные!)  
- **Diebold–Mariano тест**:  
  ```python
  dm_test(forecast_A, forecast_B, h=7, test="two_sided")
  ```
- **Сводная таблица**:
  | Модель | MAE | RMSE | MAPE | MASE | Время обучения | AutoGluon Rank |
  |--------|-----|------|------|------|----------------|----------------|
- **Ранжирование**: по MASE (основной), MAE (вторичный), p(DM) (стат. значимость)

---

### **3.8. Этап 8. Продвинутые техники**

- **Ансамблирование**:
  - Взвешенное усреднение (по MASE)  
  - Stacking с линейной мета-моделью  
  - **AutoGluon WeightedEnsemble**  
- **Обработка выбросов**:
  - Isolation Forest для фильтрации  
  - RobustScaler вместо StandardScaler  
  - Winsorization (ограничение 5%/95% перцентилями)
- **Сегментация**:
  - Кластеризация рядов (KMeans по признакам)  
  - Отдельные модели по сезонам (зима/лето) или режимам

---

### **3.9. Этап 9. Итоговый анализ и рекомендации**

1. **Ранжирование моделей** по качеству и надёжности  
2. **Сравнение AutoGluon vs кастомных моделей**:
   - Качество «из коробки»  
   - Требуемое время разработки  
   - Гибкость настройки  
   - Интерпретируемость  
3. **Анализ feature importance** топ-3 моделей  
4. **Визуализация прогноза лучшей модели** на h=1,7,30  
5. **Диагностика остатков**: есть ли автокорреляция?  
6. **AutoGluon insights**: какие внутренние модели доминируют?  
7. **Рекомендации по продакшену**:
   - Использовать AutoGluon для MVP и быстрой итерации  
   - Переходить к кастомным моделям при необходимости интерпретации или тонкой настройки  
   - Обязательно включать бейзлайны  
8. **Шаблон переобучения**:
   - Автоматическая генерация фичей  
   - Периодическая перекалибровка λ (Бокс–Кокс)  
   - Мониторинг MASE на новых данных

---

## 🔍 4. Дополнительные задания (по желанию)

1. Интеграция **SHAP-анализа** для интерпретации  
2. Сравнение **разных пресетов AutoGluon** по скорости/качеству  
3. **Прогнозные интервалы** для кастомных моделей (через квантильную регрессию)  
4. **Онлайн-обучение** (partial_fit для линейных моделей)  
5. **Мульти-сериальное моделирование** (несколько временных рядов одновременно)

---

## 📄 5. Требования к отчёту

1. Титульный лист  
2. Описание данных и источника  
3. Схема feature engineering с примерами кода  
4. Результаты валидации и разбиения  
5. Сводная таблица всех моделей по трём горизонтам  
6. Графики: прогнозы, остатки, feature importance, PDP  
7. Результаты Diebold–Mariano теста  
8. Анализ AutoGluon leaderboard  
9. Выводы и рекомендации по продакшену  
10. Рефлексия: сложности, выбор архитектуры, уроки

---

## 📊 6. Критерии оценки

| Оценка | Критерии |
|--------|---------|
| **5** | Все модели реализованы. Есть AutoGluon с разными пресетами. Полную диагностику, Diebold–Mariano, feature importance, PDP. Работающий сравнительный анализ. Выполнены доп. задания. |
| **4** | Реализованы основные модели, есть AutoGluon, метрики, диагностика. Но без PDP или одного из стратегий. |
| **3** | Построены 4–5 моделей. Есть сравнение по MAE/RMSE, но без статистических тестов или продвинутой диагностики. |
| **2** | Работа не выполнена или отсутствует содержательный анализ.

---

## 📖 7. Литература

1. Hyndman, R.J., Athanasopoulos, G. *Forecasting: Principles and Practice*, 3rd ed. — [https://otexts.com/fpp3/](https://otexts.com/fpp3/)  
2. AutoGluon Documentation — [https://auto.gluon.ai](https://auto.gluon.ai)  
3. Pedregosa et al. *Scikit-learn: Machine Learning in Python* — JMLR 2011  
4. Ke et al. *LightGBM: A Highly Efficient Gradient Boosting Decision Tree* — NeurIPS 2017  
5. Lim et al. *Temporal Fusion Transformers for Interpretable Multi-horizon Time Series Forecasting* — IJF 2021  
6. Diebold, F.X., Mariano, R.S. *Comparing Predictive Accuracy* — JBES 1995  

> 💡 Эта работа завершает цикл по **классическим и ML-методам** и формирует основу для **нейросетевого прогнозирования** (LSTM, N-BEATS, TFT) в последующих курсах.

